In [1]:
import json
import sys
sys.path.append('../../..')

In [2]:
import collections
import copy

In [4]:
# from eval_metric import normalize, compute_bleu_rouge

In [5]:
question_dict = {}
with open('/home/lq/Research/Reading-Comprehension/les-military-mrc/input/mrc_dataset_question_split/test_r0.json') as f:
    for line in f:
        sample = json.loads(line)
        question_dict[sample['question_id']] = sample['question']

In [6]:
len(question_dict)

4969

In [7]:
cnt = 0
multi_ans_question_id_list = []
for id_, question in question_dict.items():
    if (question.count('?') + question.count('？')) > 1 or (question.count('多少') + question.count('哪') + question.count('什么') + question.count('谁')) > 1:
        print(question)
        cnt += 1
        multi_ans_question_id_list.append(id_)
    
print(cnt)

C-17载重量为多少？货舱宽度为多少米？
SAR-21突击步枪它的诞生取代了哪款突击步枪？有效射程多少？
F-104是什么时候开始研制的？F-104的零长起飞系统（ZELL）”具体是什么？
东风猛士的整车高宽比为多少？这辆车的重量达到了多少吨？
黄蜂级攻击舰建造了多少艘？它拥有几具供运送航空器用的大型升降机？
B-21轰炸机将采用什么结构？B-21所搭载的核武器可能会配备什么？
短剑冲锋枪的口径是多少？且最大的特点是什么？
JAS-39C/D“鹰狮”战斗机是哪个公司研制的？其最初生产是为了替代哪两款机型？
南非獾式轮式步兵战车由哪个公司开始设计的？长约多少米吗？
M249采用了什么原理枪管可以快速的更换？该枪空枪重多少千克？
“东风-31”采用什么模式很难被敌人卫星持续跟踪？东风-31洲际导弹使用了什么运载？
055驱逐舰是哪个研究所设计的？在哪些技术上实现了突破？
“黄鼠狼”步兵战车是为协同什么主战坦克作战而设计的？约旦于2016年和2017年从德国采购了几辆“黄鼠狼”步兵战车？
准备取代MG34通用机枪的是什么型号？MG34通用机枪的射程是多少？
83式122毫米榴弹炮的射程是多少？最大射程是多少？
AIM-54“不死鸟”空空导弹的有效射程是多少公里？F-14雄猫战斗机曾经直接挂着哪种导弹充当它的替代品？
Bf109战机的首次实战就是在哪个战役中？是在上个世纪哪个年代研制的？
猎手无人机采用了什么材料？它的载弹量？
东风-16导弹的战斗部穿透深度可能接近多少米水平？导弹的技术是继承自哪款导弹的技术？
第101空中突击师主要装备什么侦察直升机？它在二战期间在哪对战多个德国装甲师并守住战地而一战成名？
AA-11导弹的射程？它是什么导弹的北约代号？
AGM158是由哪个公司研制的？它主要的缺陷是什么？
美国海岸警卫队拥有多少艘大中型巡逻舰？其中最大的是哪艘？
紫电改发动机马力提升了多少？它采用什么型发动机？
“好人理查德”号执行对地攻击任务时可以搭载多少架CH-46海骑士直升机？它退出现役后转入了哪支舰队？
RS-26“边界”导弹西方按其数十年来形成的传统将其命名为什么名称？RS-26“边界”其发射车重量为多少吨？
FR-F2狙击步枪枪身长度为多少？FR-F2狙击步枪有什么优点？
沱江级有哪些特点？它为什么被外界称为航母杀手？
德国在20世纪初至第一次世界大战爆发前

In [8]:
multi_ans_question_id_list[0]

'Q_34851qyhyc'

In [9]:
with open('../les_mrc/bert_baseline/models/les_wwm_ques_split_span_mask/nbest_predictions_checkpoint_test.json') as f:
    all_nbest_json = json.loads(f.read())

In [10]:
all_samples = collections.defaultdict(list)
need_skippeed_list = [',', '.', '。']
for qas_id, nbest_json in all_nbest_json.items():
    text = ''
    prob = 0.0
    logit = 0.0
    for entry in nbest_json:
        if entry['text'].strip() and entry['text'] not in need_skippeed_list:
            text = entry['text'].strip()
            logit = entry['start_logit'] + entry['end_logit']
            prob = entry['probability']
            break
    all_samples[qas_id.split('##')[0]].append([text, logit, prob])
all_predictions = {}
multi_pred = []
cnt = 0
for question_id, sample in all_samples.items():
    sample = sorted(sample, key=lambda x: x[1], reverse=True)
    all_predictions[question_id] = sample[0][0]
#     print(sample[0][2])
    # 简单的多答案选择模块
    if question_id in multi_ans_question_id_list:
#         for i in range(1, 5):
#             ans1 = normalize([sample[0][0]])
#             ans2 = normalize([sample[i][0]])
#         if sample[0][0] == sample[1][0]:
#             continue
        all_predictions[question_id] = all_predictions[question_id] + '#' + sample[1][0]
        print(all_predictions[question_id])
        cnt += 1
#     if sample[1][0] != '' and sample[1][2] > 0.5:
#         # 有可能具有多答案
#         if sample[0][0] == sample[1][0]:
#             continue
#         if sample[1][0] in sample[0][0] or sample[0][0] in sample[1][0]:
#             continue
#         ans1 = normalize([sample[0][0]])
#         ans2 = normalize([sample[1][0]])
#         bleu_rouge = compute_bleu_rouge({'item': ans1}, {'item': ans2})
#         if bleu_rouge['Bleu-4'] > 0.2 or bleu_rouge['Rouge-L'] > 0.2:
#             continue
#         print('{} have multi-ans, take care of it'.format(question_id))
#         multi_pred.append(question_id)
#         cnt += 1
#         all_predictions[question_id] = all_predictions[question_id] + '#' + sample[1][0]
        
print(cnt)

货舱宽度为5.49米#C-17货舱宽度为5.49米
它的诞生取代了新加坡军队即将到达服役期限的M16突击步枪#885米
1951年#在F-104战斗机机腹下方安装了一台固体火箭发动机。
整车高宽比为0.91#整车高宽比为0.91
12艘#两具
可能会配备现在正在开发的远程防区外武器(LRSO)#飞翼结构
射速快，后座力低#11.43毫米
萨博(SAAB)公司#Saab-35“龙”战斗机和Saab-37“雷”战斗机
长约8米#南非桑达克·奥斯特瑞尔·比派克公司
采用了气动、气冷原理#采用气动、气冷原理
采用公路机动模式#卡车拖挂式
中国船舶重工集团701研究所设计#突破了大型舰艇总体设计、信息集成、总装建造等一系列关键技术
25辆#为协同“豹”1主战坦克作战而设计的
MG42#800米以上
18公里#15.6公里
MIM-23“霍克”地空导弹#184公里
西班牙内战中#30年代
全新的航空复合材料#约为8吨）
40-50米#东风11导弹的技术
比利时小镇巴斯通#OH-58D型“基奥瓦”侦察直升机
R-73导弹#16千米
其主要缺陷一直是效费比问题#小型喷气发动机提供动力，飞行速度只有0、8马赫。
240艘#诺斯罗普格鲁门公司生产的“传奇”级（Legend）国家安全舰。
誉21型发动机#190
布雷默顿预备役舰队#42架
SS-X-29#其发射车重量为80吨
1138毫米#射击稳定性好，攻击力强
主因为舰上搭载16枚杀伤力强大的雄风系列反舰导弹。#匿踪能力，速度快特性，并能够携带8枚雄风二型与8枚雄风三型反舰导弹，火力强大。
8门双联装380mm/52倍径主炮#17艘
4.42千克#3.31千克
3架美制C-130运输机#123架
1000吨#两座海拉姆防空导弹和8枚鱼叉导弹
莫斯科“劳动旗帜”工厂（现米格公司第二生产中心）#63吨
一个中国原则和中美三个联合公报规定#三个联合公报
1984-1991年#"特拉法加"级攻击型核潜艇是英国海军主战攻击型核潜艇，是英国第3代攻击型核潜艇“敏捷”级
1.78马赫#F-22机身蒙皮全都是高强度、耐高温的BMI复合材料
8马赫，约9800公里每小时#2.5马赫
B-2隐身战略轰炸机#61米
20毫米#1541架
霹雳15远程空空导弹#2架
25发一小节的分段式结构#1000米以内的集团目标
134米#134米
14颗#全枪重量为3

In [11]:
with open('multi_ans_test_0914.json', "w") as writer:
    writer.write(json.dumps(all_predictions, indent=4, ensure_ascii=False) + "\n")

In [15]:
all_samples['Q_21851qshyg']

[['1951年开始研制', 11.737119197845459, 0.7095357522128155],
 ['1951年', 2.258337080478668, 1.611248185293107e-05],
 ['1954年', -5.653919458389282, 5.909226453829384e-09],
 ['1963年', 2.561307668685913, 0.0014150720752246747],
 ['1958年初', -0.598359614610672, 1.379661289566476e-06]]

In [17]:
question_dict['Q_33788kyqdg']

'83式122毫米榴弹炮的射程是多少？最大射程是多少？'